# Chapter 1

This chapter focuses on explaining the basics of option pracing and how to do it using monte carlo techniques.

Options are a type of financial instrument that rely on the price of another asset. These type of instruments are the so called derivatives. Derivatives price is not a straight forward number, they have to be calculated with a model that takes into consideration different variables.

In this chapter the vanilla call option will be priced through a montecarlo simulation.

The purpose of this notebook is to repeat the steps and the code developed in C++ Design Patterns and Derivatives Pricing, written by Mark S. Joshi, but in python, also solving the different exercises.

If more information is required about what a call option is check https://en.wikipedia.org/wiki/Call_option

Also an excellent book for learning the basics of derivatives pricing is Options, Futures, and Other Derivatives by  John C. Hull.

## A simple implementation of a Monte Carlo call option pricer

### Imports

In [5]:
from random import random
from math import exp, sqrt, log

### Basic functions

In [9]:
def SimpleMonteCarlo1(Expiry,
                     Strike,
                     Spot,
                     Vol,
                     r,
                     NumberOfPaths):
    
    variance = Vol*Vol*Expiry
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    movedSpot = Spot*exp(r*Expiry + itoCorrection)
    runningSum = 0
    for i in range(NumberOfPaths):
        thisGaussian = GetOneGaussianByBoxMuller()
        thisSpot = movedSpot*exp(rootVariance*thisGaussian)
        thisPayoff = thisSpot - Strike
        if thisPayoff < 0 : 
            thisPayoff = 0
        runningSum += thisPayoff
    
    mean = runningSum / NumberOfPaths
    mean *= exp(-r*Expiry)
    return mean            

In [6]:
def GetOneGaussianBySummation():
    result = 0
    for j in range(12):
        result += random()
    result -= 6
    return result

def GetOneGaussianByBoxMuller():
    sizeSquared = 1
    while sizeSquared >= 1:
        x = 2*random() - 1
        y = 2*random() - 1
        sizeSquared = x*x + y*y
    
    result = x*sqrt(-2*log(sizeSquared)/sizeSquared)
    return result

### Time to test

In the original book there is another function (main), to introduce the data with the console, but since we are using a notebook it does not make sense to use this type of input function, instead we will call the function with some parameters and test it.

The parameters that we will input are the ones in the example of calculating the price of a call option in Hull.

They are the following:
* Expiry: 6 months (0.5 years)
* Strike: 40
* Spot: 42
* Vol: 0.2
* r: 0.1

The price of the call option in the example calculated with the Black Scholes model is 4.76.

We will with different NumberOfPaths and see how accurate is the result.

In [19]:
for i in range(1,200,10):
    i = i*i
    callPrice = SimpleMonteCarlo1(0.5,
                     40,
                     42,
                     0.2,
                     0.1,
                     i)
    print(f'Call price: {round(callPrice,2)}, for {i} paths')

Call price: 4.87, for 1 paths
Call price: 4.74, for 121 paths
Call price: 4.52, for 441 paths
Call price: 4.88, for 961 paths
Call price: 4.85, for 1681 paths
Call price: 4.77, for 2601 paths
Call price: 4.79, for 3721 paths
Call price: 4.68, for 5041 paths
Call price: 4.73, for 6561 paths
Call price: 4.76, for 8281 paths
Call price: 4.65, for 10201 paths
Call price: 4.67, for 12321 paths
Call price: 4.82, for 14641 paths
Call price: 4.69, for 17161 paths
Call price: 4.76, for 19881 paths
Call price: 4.73, for 22801 paths
Call price: 4.81, for 25921 paths
Call price: 4.79, for 29241 paths
Call price: 4.77, for 32761 paths
Call price: 4.76, for 36481 paths


As we can see our simple routine is able to provide a close price to the ideal price calculated by the Black Scholes Model

### Critiques to the simple Monte Carlo routine

Our routine can provide a decent estimation of the price, however it is slower than in C++ (Python is not a Ferrari... , specially when you do not use Numpy or Pandas for fast computing).

Also other considerations can be done to our routine. The purpose of the code is to be reusable for other projects, if we want to implement similar projects using part our code will mean rewriting it. This is not optimal. Imagine our boss (or evil boss as mr Joshi says) now wants to do puts as well as calls, put the standard error, use anthithetic sampling, do a digital call pricer ...

To add reusability to our code we will use classes and objects, that will allow us to implement new features without touching the existing code, while using most of what we have already built.

### Exercises

**Exercise 1.1** Modify the program given to price puts.

Puts are the opposite to calls, so we have to midify only one line of the SimpleMonteCarlo 1, instead of *thisPayoff = thisSpot - Strike* we will put *thisPayoff = Strike - thisSpot* .

We will use the previous example, the put price should be 0.81 (way lower than the call since it is out of the money)

In [48]:
def SimpleMonteCarlo1(Expiry,
                     Strike,
                     Spot,
                     Vol,
                     r,
                     NumberOfPaths):
    
    variance = Vol*Vol*Expiry
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    movedSpot = Spot*exp(r*Expiry + itoCorrection)
    runningSum = 0
    for i in range(NumberOfPaths):
        thisGaussian = GetOneGaussianByBoxMuller()
        thisSpot = movedSpot*exp(rootVariance*thisGaussian)
        thisPayoff = Strike - thisSpot
        if thisPayoff < 0 : 
            thisPayoff = 0
        runningSum += thisPayoff
    
    mean = runningSum / NumberOfPaths
    mean *= exp(-r*Expiry)
    return mean  

In [49]:
for i in range(1,200,10):
    i = i*i
    callPrice = SimpleMonteCarlo1(0.5,
                     40,
                     42,
                     0.2,
                     0.1,
                     i)
    print(f'Put price: {round(callPrice,2)}, for {i} paths')

Put price: 2.25, for 1 paths
Put price: 0.96, for 121 paths
Put price: 0.62, for 441 paths
Put price: 0.77, for 961 paths
Put price: 0.86, for 1681 paths
Put price: 0.78, for 2601 paths
Put price: 0.78, for 3721 paths
Put price: 0.8, for 5041 paths
Put price: 0.82, for 6561 paths
Put price: 0.8, for 8281 paths
Put price: 0.82, for 10201 paths
Put price: 0.79, for 12321 paths
Put price: 0.82, for 14641 paths
Put price: 0.82, for 17161 paths
Put price: 0.83, for 19881 paths
Put price: 0.8, for 22801 paths
Put price: 0.83, for 25921 paths
Put price: 0.82, for 29241 paths
Put price: 0.81, for 32761 paths
Put price: 0.8, for 36481 paths


Our code has given us a well approximation to the put option price we were expecting.

**Exercise 1.2** Modify the program given to price double digitals.

This part is a bit more tricky. A double digital option is an financial instrument that pays 1 if the underlaying asset price is between to strike prices, and 0 if not. We will modify the code to input 2 strike prices and then add the part of evaluating if it is between the two frontiers.

In [43]:
def SimpleMonteCarlo1(Expiry,
                     StrikeUpper,
                     StrikeLower,
                     Spot,
                     Vol,
                     r,
                     NumberOfPaths):
    
    variance = Vol*Vol*Expiry
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    movedSpot = Spot*exp(r*Expiry + itoCorrection)
    runningSum = 0
    # We add this snippet in case the user has put in an incorrect order the Upper and Lower strike prices
    if StrikeLower > StrikeUpper:
        aux = StrikeLower
        StrikeLower = StrikeUpper
        StrikeUpper = aux
        del aux
    else:
        pass
    # End of the snippet code
    for i in range(NumberOfPaths):
        thisGaussian = GetOneGaussianByBoxMuller()
        thisSpot = movedSpot*exp(rootVariance*thisGaussian)
        if thisSpot <= StrikeUpper and thisSpot >= StrikeLower:
            thisPayoff = 1            
        else:
            thisPayoff = 0        
        runningSum += thisPayoff
    
    mean = runningSum / NumberOfPaths
    mean *= exp(-r*Expiry)
    return mean

A good way to know if our code is correct is to check it with an example.

That example has te following data:    
* Number of Paths: 10000000
* Underlying:      100
* Lower Strike:    100
* Upper Strike:    120
* Risk-Free Rate:  0.05
* Volatility:      0.2
* Maturity:        1
* Result Price:   0.32009

The example comes from: https://www.quantstart.com/articles/Double-digital-option-pricing-with-C-via-Monte-Carlo-methods

In [46]:
for i in range(1,200,10):
    i = i*i
    DoubleDigitalOption = SimpleMonteCarlo1(1,
                     120,
                     100,
                     100,
                     0.2,
                     0.05,
                     i)
    print(f'Double digital option: {round(DoubleDigitalOption,2)}, for {i} paths')

Double digital option: 0.0, for 1 paths
Double digital option: 0.32, for 121 paths
Double digital option: 0.3, for 441 paths
Double digital option: 0.32, for 961 paths
Double digital option: 0.33, for 1681 paths
Double digital option: 0.32, for 2601 paths
Double digital option: 0.33, for 3721 paths
Double digital option: 0.31, for 5041 paths
Double digital option: 0.33, for 6561 paths
Double digital option: 0.33, for 8281 paths
Double digital option: 0.32, for 10201 paths
Double digital option: 0.32, for 12321 paths
Double digital option: 0.32, for 14641 paths
Double digital option: 0.32, for 17161 paths
Double digital option: 0.32, for 19881 paths
Double digital option: 0.32, for 22801 paths
Double digital option: 0.32, for 25921 paths
Double digital option: 0.32, for 29241 paths
Double digital option: 0.32, for 32761 paths
Double digital option: 0.32, for 36481 paths


The result is very similar to the example so we can suppose it is good.

**Exercise 1.3** Change the program so that the user inputs a string which specifies the option pay-off.

We will suppose that we only have two options, call and put, we will also suppose that the user will only input 'call' or 'put' (to avoid the error handling part).

In [53]:
def SimpleMonteCarlo1(Expiry,
                     Strike,
                     Spot,
                     Vol,
                     r,
                     NumberOfPaths,
                     OptionType):
    
    variance = Vol*Vol*Expiry
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    movedSpot = Spot*exp(r*Expiry + itoCorrection)
    runningSum = 0
    if OptionType == 'call':
        for i in range(NumberOfPaths):
            thisGaussian = GetOneGaussianByBoxMuller()
            thisSpot = movedSpot*exp(rootVariance*thisGaussian)
            thisPayoff = thisSpot - Strike
            if thisPayoff < 0 : 
                thisPayoff = 0
            runningSum += thisPayoff
            
    elif OptionType == 'put':
        for i in range(NumberOfPaths):
            thisGaussian = GetOneGaussianByBoxMuller()
            thisSpot = movedSpot*exp(rootVariance*thisGaussian)
            thisPayoff = Strike - thisSpot
            if thisPayoff < 0 : 
                thisPayoff = 0
            runningSum += thisPayoff
        
    
    mean = runningSum / NumberOfPaths
    mean *= exp(-r*Expiry)
    return mean    

We add the if clause to be faster and avoid checking that condition inside the loop. Let´s try with some examples.

In [55]:
for i in range(1,200,10):
    i = i*i
    callPrice = SimpleMonteCarlo1(0.5,
                     40,
                     42,
                     0.2,
                     0.1,
                     i,
                     'call')
    print(f'Call price: {round(callPrice,2)}, for {i} paths')

Call price: 7.86, for 1 paths
Call price: 4.91, for 121 paths
Call price: 4.5, for 441 paths
Call price: 4.66, for 961 paths
Call price: 4.77, for 1681 paths
Call price: 4.67, for 2601 paths
Call price: 4.68, for 3721 paths
Call price: 4.74, for 5041 paths
Call price: 4.78, for 6561 paths
Call price: 4.73, for 8281 paths
Call price: 4.75, for 10201 paths
Call price: 4.73, for 12321 paths
Call price: 4.75, for 14641 paths
Call price: 4.76, for 17161 paths
Call price: 4.75, for 19881 paths
Call price: 4.76, for 22801 paths
Call price: 4.79, for 25921 paths
Call price: 4.83, for 29241 paths
Call price: 4.73, for 32761 paths
Call price: 4.82, for 36481 paths


In [54]:
for i in range(1,200,10):
    i = i*i
    callPrice = SimpleMonteCarlo1(0.5,
                     40,
                     42,
                     0.2,
                     0.1,
                     i,
                     'put')
    print(f'Put price: {round(callPrice,2)}, for {i} paths')

Put price: 0.0, for 1 paths
Put price: 1.0, for 121 paths
Put price: 0.88, for 441 paths
Put price: 0.83, for 961 paths
Put price: 0.84, for 1681 paths
Put price: 0.81, for 2601 paths
Put price: 0.79, for 3721 paths
Put price: 0.77, for 5041 paths
Put price: 0.82, for 6561 paths
Put price: 0.86, for 8281 paths
Put price: 0.82, for 10201 paths
Put price: 0.81, for 12321 paths
Put price: 0.8, for 14641 paths
Put price: 0.8, for 17161 paths
Put price: 0.82, for 19881 paths
Put price: 0.81, for 22801 paths
Put price: 0.81, for 25921 paths
Put price: 0.8, for 29241 paths
Put price: 0.81, for 32761 paths
Put price: 0.82, for 36481 paths


**Exercise 1.4** Identify as many classes as you can in the evil boss's list of demands.

Some examples would be the parent class PayOff the subclasses CallPayOff, PutPayOff ...